In [1]:
import os
import shutil
import random
import librosa
import numpy as np
from glob import glob
from scipy import signal
from sklearn.svm import SVC
from IPython import display
from IPython.display import SVG
import matplotlib.pyplot as plt
import keras
from keras import backend as K
from keras.models import load_model
#from keras.utils import model_to_dot
from keras.utils import to_categorical
from keras.models import Model, load_model
from keras.layers import Input, Conv2D, Dropout, MaxPooling2D, AveragePooling2D, Flatten, Dense, Activation, BatchNormalization
from python_speech_features import mfcc

#For fft spectrum.
import sigproc
import constants as c
from scipy.signal import lfilter, butter


os.environ['CUDA_VISIBLE_DEVICES'] = '1'
#import psutil
#p = psutil.Process()
#p.cpu_affinity([0,1,2,5,9,13,17,18,19,20,23,27,28,29,30,31])

Using TensorFlow backend.


In [ ]:
batch_size = 200
no_batches = 0
batch_sum = 0
curr_size = batch_size

while (batch_sum + curr_size) < 1000000:
    batch_sum += curr_size
    no_batches += 1
    if no_batches%10 == 0:
        curr_size += 1

In [ ]:
train_params = {'dim': (512,299),
                'batch_size': 256,
                'n_classes': 5994,
                'n_channels': 1,
                'shuffle': True}

In [ ]:
def rename_folders():
    '''
    Rename all speaker directories in numeric order to represent the speaker's ID.
    '''    
    speakers = glob("/data/techresearch/Murtaza/vox2/dev/aac/*")    
    #Make sure only valid speaker directories are in list.
    for speaker in speakers:    
        assert os.path.isdir(speaker)
        assert 'id' in speaker
    for i in range(0,len(speakers)):
        old_path = speakers[i] 
        new_path = os.path.split(speakers[i])[0] + '/' + str(i) 
        os.rename(old_path, new_path)
    print('Folders renamed: ' + str(len(speakers)))
    return

In [ ]:
def rearrange_audios():
    '''
    Rearrange all the files in the dataset so that all audios
    of a particular speaker are in the same directory.
    '''
    speakers = glob("/data/techresearch/Murtaza/vox2/dev/aac/*/") 
    counter = 0
    for speaker in speakers:
        subdirs = os.listdir(speaker)                         #List subdirectories for a particular speaker.
        counter += 1
        print(counter)
        for j in range(len(subdirs)):
            subdir = subdirs[j]
            if os.path.isdir(speaker + subdir):               #Check if its a directory and not a file.
                audio_files = os.listdir(speaker + subdir)    #List files for a particular speaker in a particular subfolder.
                for audio_file in audio_files:
                    if audio_file.endswith('.m4a'):          #Check if file is audio file and not a text file or something unwanted.
                        old_path = speaker + subdir + '/' + audio_file 
                        new_path = speaker + str(j) + '_' + audio_file
                        os.rename(old_path, new_path)         #Move the audiofile to main speaker directory.
                shutil.rmtree(speaker + subdir)               #Remove subdirectory.

In [ ]:
def rename_audio_files():
    '''
    Rename all audios to include speaker ID in their name.
    '''
    audio_files = glob("/data/techresearch/Murtaza/vox2/dev/aac/*/*.m4a")
    for path in audio_files:
        label = os.path.split(os.path.split(path)[0])[1]
        path_pre = os.path.split(path)[0]
        path_post = os.path.split(path)[1]
        new_path = path_pre + '/' + label + '_' + path_post
        os.rename(path, new_path)
        

In [ ]:
def create_validation_split():
    '''
    Set aside a small portion of the data for validation.
    '''
    val_size = 8000
    audio_files = glob("/data/techresearch/Murtaza/vox2/dev/aac/*/*.m4a")
    random.shuffle(audio_files)
    val_files = audio_files[:8000]
    for path in val_files:
        pre_path = '/data/techresearch/Murtaza/vox2/val/aac/'
        post_path = os.path.split(path)[1]
        new_path = '/data/techresearch/Murtaza/vox2/val/aac/' + post_path
        os.rename(path, new_path)

In [2]:
#Functions for fft spectrum. Copied from 'https://github.com/linhdvu14/vggvox-speaker-identification/blob/master'
def remove_dc_and_dither(sin, sample_rate):
	if sample_rate == 16e3:
		alpha = 0.99
	elif sample_rate == 8e3:
		alpha = 0.999
	else:
		print("Sample rate must be 16kHz or 8kHz only")
		exit(1)
	sin = lfilter([1,-1], [1,-alpha], sin)
	dither = np.random.random_sample(len(sin)) + np.random.random_sample(len(sin)) - 1
	spow = np.std(dither)
	sout = sin + 1e-6 * spow * dither
	return sout
        

def normalize_frames(m,epsilon=1e-12):
	return np.array([(v - np.mean(v)) / max(np.std(v),epsilon) for v in m])


def get_fft_spectrum(signal, buckets=None):
	#signal = load_wav(filename,c.SAMPLE_RATE)
	signal *= 2**15

	# get FFT spectrum
	signal = remove_dc_and_dither(signal, c.SAMPLE_RATE)
	#signal = sigproc.preemphasis(signal, coeff=c.PREEMPHASIS_ALPHA)
	frames = sigproc.framesig(signal, frame_len=c.FRAME_LEN*c.SAMPLE_RATE, frame_step=c.FRAME_STEP*c.SAMPLE_RATE, winfunc=np.hamming)
	fft = abs(np.fft.fft(frames,n=c.NUM_FFT))
	fft_norm = normalize_frames(fft.T)

	# truncate to max bucket sizes
	#rsize = max(k for k in buckets if k <= fft_norm.shape[1])
	#rstart = int((fft_norm.shape[1]-rsize)/2)
	#out = fft_norm[:,rstart:rstart+rsize]

	#return out
	return fft_norm

In [3]:
'''Note: Study required for optimal spectogram.'''

def audio_to_image(audio, sr):                            
    '''
    Convert an audio to its spectrogram.
    '''
    #_,_,spectrogram = signal.spectrogram(audio, sr)
    #audio_mfcc = librosa.feature.mfcc(y=audio, sr=16000,  n_mfcc=60)
    #audio_mfcc2 = mfcc(signal=audio, samplerate=16000, nfft=512, winlen=0.025, winstep=0.009, nfilt=400, numcep = 300, winfunc=np.hamming)
    audio_mfcc3 = get_fft_spectrum(audio)
    return audio_mfcc3

In [ ]:
def get_label(path):
    '''
    Extract label of audio from the given path.
    '''
    label = os.path.split(os.path.split(path)[0])[1]
    label = int(label)
    return label

In [4]:
def get_label_enroll(path):
    '''
    Extract label of audio from 
    the given path.
    '''
    label = path.split('/')[-1]
    label = label.replace('.wav', '')
    label = int(label)
    return label

In [5]:
def get_label_eval(path):
    '''
    Extract label of audio from 
    the given path.
    '''
    label = path.split('_')[-2]
    label = int(label)
    return label

In [ ]:
def get_label_val_vox2(path):
    '''
    Extract label of audio from 
    the given path.
    '''
    label = path.split('/')[-1]
    label = label.split('_')[-3]
    label = int(label)
    return label

In [ ]:
'''
def load_training_dataset_old(root_path, no_speakers):
    '''
    #Returns the complete dataset for training and the associated labels.
    #Note that the audios are converted to images.
    '''
    audio_rate = 16000
    no_seconds = 3
    x_train = []
    y_train = []
    
    speakers = glob(root_path + '/*/')    #List of speaker directories.
    speakers = speakers[:no_speakers]
    
    for j,speaker in enumerate(speakers):
        audio_files = glob(speaker + '*.wav')
        
        for audio_file in audio_files:            
            label = get_label(audio_file)
            audio_data, _ = librosa.load(sr=audio_rate, mono=True, path=audio_file)
            
            assert _ == audio_rate
            no_clips = len(audio_data) // (audio_rate*no_seconds)
            
            for i in range(0, no_clips):                                                                #4 sec clips
                audio_clip = audio_data[(audio_rate*no_seconds)*i:(audio_rate*no_seconds)*(i+1)]
                image = audio_to_image(audio_clip, audio_rate)
                x_train.append(image)
                y_train.append(label)
        
        print('No. speakers loaded: ' + str(j + 1))    
    return x_train, y_train
'''

In [ ]:
def create_training_dataset_vox1():
    '''
    Create the complete dataset for training and the associated labels. 
    The dataset is created and saved in subsets due to limitaion of RAM.
    Note that the audios are converted to images.
    '''
    
    audio_files = glob("/data/techresearch/Murtaza/vox1/dev/wav/*/*.wav")
    random.shuffle(audio_files)
    
    subset_size = 9000
    total_subsets = 16
    audio_rate = 16000
    no_seconds = 3
    count = 0
    for j in range(total_subsets):
        x_train = []
        y_train = []
        
        audio_subset = audio_files[ (subset_size*j) : (subset_size*(j+1)) ]
        print(str((subset_size*j)) + ' to ' + str(subset_size*(j+1)))

        for audio_file in audio_subset:
            count+=1
            print(count)
            label = get_label(audio_file)
            audio_data, _ = librosa.load(sr=audio_rate, mono=True, path=audio_file)

            assert _ == audio_rate
            no_clips = len(audio_data) // (audio_rate*no_seconds)

            for i in range(0, no_clips):                                                                #4 sec clips
                audio_clip = audio_data[(audio_rate*no_seconds)*i:(audio_rate*no_seconds)*(i+1)]
                image = audio_to_image(audio_clip, audio_rate)
                x_train.append(image)
                y_train.append(label)
    
        #Convert to numpy
        x_train = np.asarray(x_train)
        y_train = np.asarray(y_train)
        #Save the numpy data
        np.save('/data/techresearch/Murtaza/mfcc3/x_train_' + str(j) + '.npy', x_train)
        np.save('/data/techresearch/Murtaza/mfcc3/y_train_' + str(j) + '.npy', y_train)
        
    return

In [6]:
def create_enrollment_dataset():
    '''
    Returns the complete dataset for enrollment and the associated labels.
    Note that the audios are converted to images.
    '''
    audio_rate = 16000
    no_seconds = 3
    x_train = []
    y_train = []
    
    audio_files = glob('/home/techresearch/rnn/app_enrollment/data/enrollment/*.wav')
    random.shuffle(audio_files)
    for j, audio_file in enumerate(audio_files):            
        label = get_label_enroll(audio_file)
        audio_data, _ = librosa.load(sr=audio_rate, mono=True, path=audio_file)

        assert _ == audio_rate
        no_clips = len(audio_data) // (audio_rate*no_seconds)

        for i in range(0, no_clips):                                                                #4 sec clips
            audio_clip = audio_data[(audio_rate*no_seconds)*i:(audio_rate*no_seconds)*(i+1)]
            image = audio_to_image(audio_clip, audio_rate)
            x_train.append(image)
            y_train.append(label)
        print('No. Speakers loaded: ' + str(j + 1))
    
    #Convert to numpy
    x_train = np.asarray(x_train)
    y_train = np.asarray(y_train)
    
    return x_train, y_train

In [7]:
def create_evaluation_dataset():
    '''
    Returns the complete dataset for evaluation and the associated labels.
    Note that the audios are converted to images.
    '''
    audio_rate = 16000
    no_seconds = 3
    x_train = []
    y_train = []
    
    audio_files = glob('/home/techresearch/rnn/app_enrollment/data/evaluation_original/*.wav')
    random.shuffle(audio_files)
    for audio_file in audio_files:            
        label = get_label_eval(audio_file)
        audio_data, _ = librosa.load(sr=audio_rate, mono=True, path=audio_file)

        assert _ == audio_rate
        no_clips = len(audio_data) // (audio_rate*no_seconds)

        for i in range(0, 1):                                                                #3 sec clips
            audio_clip = audio_data[(audio_rate*no_seconds)*i:(audio_rate*no_seconds)*(i+1)]
            image = audio_to_image(audio_clip, audio_rate)
            x_train.append(image)
            y_train.append(label)
    
    #Convert to numpy
    x_train = np.asarray(x_train)
    y_train = np.asarray(y_train)
    
    return x_train, y_train

In [ ]:
audio_files = glob('/home/techresearch/rnn/app_enrollment/data/evaluation_original/*.wav')

In [ ]:
audio_files

In [ ]:
def create_val_dataset_vox2():
    '''
    Create the complete dataset for validation and the associated labels.
    Note that the audios are converted to images.
    '''
    audio_rate = 16000
    no_seconds = 3
    x_train = []
    y_train = []
    count = 0
    
    audio_files = glob("/data/techresearch/Murtaza/vox2/val/wav/*.wav")
    random.shuffle(audio_files)
    for j, audio_file in enumerate(audio_files):
        label = get_label_val_vox2(audio_file)
        audio_data, _ = librosa.load(sr=None, mono=True, path=audio_file)

        assert _ == audio_rate
        no_clips = len(audio_data) // (audio_rate*no_seconds)

        if len(audio_data) >= (audio_rate*no_seconds):          
            audio_clip = audio_data[0 : audio_rate*no_seconds]
            image = audio_to_image(audio_clip, audio_rate)
            x_train.append(image)
            y_train.append(label)
            
        count +=1
        if count%100 == 0:
            print(count)
    
    #Convert to numpy
    x_train = np.asarray(x_train)
    y_train = np.asarray(y_train)
    #Save the numpy data
    np.save('/data/techresearch/Murtaza/vox2/val/x_val' + '.npy', x_train)
    np.save('/data/techresearch/Murtaza/vox2/val/y_val' + '.npy', y_train)
    
    return

In [ ]:
'''
def vox2_iterative_generative_training(model):

    
    audio_files = glob("/data/techresearch/Murtaza/vox2/dev/aac/*/*.m4a")
    random.shuffle(audio_files)
    
    no_epochs = 15
    subset_size = 5000
    total_subsets = len(audio_files)//subset_size
    audio_rate = 16000
    no_seconds = 3
    count = 0
    for epoch in range(0,no_epochs):
        
        for j in range(0,total_subsets):
            x_train = []
            y_train = []
            count = 0
            
            audio_subset = audio_files[ (subset_size*j) : (subset_size*(j+1)) ]
            print(str((subset_size*j)) + ' to ' + str(subset_size*(j+1)))

            for audio_file in audio_subset:
                count +=1
                print(count)
                label = get_label(audio_file)
                audio_data, _ = librosa.load(sr=audio_rate, mono=True, path=audio_file)
        
                assert _ == audio_rate

                if len(audio_data)/audio_rate >= no_seconds:
                    start = random.randint(0, len(audio_data) - (audio_rate*no_seconds) )
                    end = start + (audio_rate*no_seconds)
                    audio_clip = audio_data[start : end]
                    image = audio_to_image(audio_clip, audio_rate)
                    x_train.append(image)
                    y_train.append(label)
                    count += 1
                    print(count)

            #Convert to numpy
            x_train = np.asarray(x_train)
            y_train = np.asarray(y_train)
            
            #Reshape x_train to have 1 channel.
            x_train = np.reshape(x_train, [-1,512,299,1])
            #Onehot encoding for y_train.
            y_train = to_categorical(y_train, num_classes=5993)
            
            print('Epoch: ' + str(epoch))
            model.fit(x=x_train, y=y_train, batch_size=32, epochs=1, validation_split=0.03)
             
    
    return
'''

def vox2_iterative_generative_training(model, feature_model1, feature_model2):
    '''
    
    '''
    audio_files = glob("/data/techresearch/Murtaza/vox2/dev/wav/*/*.wav")
    random.shuffle(audio_files)
    
    no_epochs = 15
    subset_size = 5000
    total_subsets = len(audio_files)//subset_size
    audio_rate = 16000
    no_seconds = 3
    
    for epoch in range(0,no_epochs):
        
        for j in range(0,total_subsets):
            x_train = []
            y_train = []
            
            audio_subset = audio_files[ (subset_size*j) : (subset_size*(j+1)) ]
            print(str((subset_size*j)) + ' to ' + str(subset_size*(j+1)))

            for audio_file in audio_subset:
                label = get_label(audio_file)
                #audio_data, _ = librosa.load(sr=audio_rate, mono=True, path=audio_file)
                audio_data, _ = librosa.load(sr=None, mono=True, path=audio_file)
                #audio_data = AudioSegment.from_file(audio_file)
                #audio_data = audio_data.get_array_of_samples()
                #audio_data = np.array(audio_data)

                #assert _ == audio_rate

                if len(audio_data)/audio_rate >= no_seconds:
                    start = random.randint(0, len(audio_data) - (audio_rate*no_seconds) )
                    end = start + (audio_rate*no_seconds)
                    audio_clip = audio_data[start : end]
                    image = audio_to_image(audio_clip, audio_rate)
                    x_train.append(image)
                    y_train.append(label)

            #Convert to numpy
            x_train = np.asarray(x_train)
            y_train = np.asarray(y_train)
            
            #Reshape x_train to have 1 channel.
            x_train = np.reshape(x_train, [-1,512,299,1])
            #Onehot encoding for y_train.
            y_train = to_categorical(y_train, num_classes=5994)
            
            print('Epoch: ' + str(epoch))
            model.fit(x=x_train, y=y_train, batch_size=32, epochs=1, validation_split=0.03)
            
            if (j+1)%40 == 0:
                
                x_train = np.load('/data/techresearch/Murtaza/vox2/val/x_val.npy')
                y_train = np.load('/data/techresearch/Murtaza/vox2/val/y_val.npy')
                #Reshape x_train to have 1 channel.
                x_train = np.reshape(x_train, [-1,512,299,1])
                #Onehot encoding for y_train.
                y_train = to_categorical(y_train)
                eval_result = model.evaluate(x=x_train, y=y_train) 
                
                file = open('train_log.txt', 'a')
                file.write('Epoch: ' + str(epoch) + '\n')
                file.write(str((subset_size*j)) + ' to ' + str(subset_size*(j+1)) + '\n')
                file.write(str(eval_result))
                file.write('\n\n')
                file.close
                
                model.save('/data/techresearch/Murtaza/vox2/dev/weights/model_' + str(epoch+1) + '_' + str(j+1) + '.h5')
                feature_model1.save('/data/techresearch/Murtaza/vox2/dev/weights/featuremodel1_' + str(epoch+1) + '_' + str(j+1) + '.h5')
                feature_model2.save('/data/techresearch/Murtaza/vox2/dev/weights/featuremodel2_' + str(epoch+1) + '_' + str(j+1) + '.h5')
                model.save_weights('/data/techresearch/Murtaza/vox2/dev/weights/weights_' + str(epoch+1) + '_' + str(j+1) + '.h5')             
    
    return


In [ ]:
epoch = 5
subset_size = 5000
j = 5
eval_result = [0.0045, 0.0]

In [ ]:
def train_log():
    file = open('train_log.txt', 'a')
    file.write('Epoch: ' + str(epoch) + '\n')
    file.write(str((subset_size*j)) + ' to ' + str(subset_size*(j+1)) + '\n')
    file.write(str(eval_result))
    file.write('\n\n')
    file.close

In [ ]:
train_log()

In [ ]:
x_train = np.load('/data/techresearch/Murtaza/vox2/val/x_val.npy')
y_train = np.load('/data/techresearch/Murtaza/vox2/val/y_val.npy')
#Reshape x_train to have 1 channel.
x_train = np.reshape(x_train, [-1,512,299,1])
#Onehot encoding for y_train.
y_train = to_categorical(y_train)
eval_result = model.evaluate(x=x_train, y=y_train) 

In [ ]:
audio_files = glob("/data/techresearch/Murtaza/vox2/val/wav/*.wav")

In [ ]:
from pydub import AudioSegment
audio = AudioSegment.from_file(audio_file)
audio = audio.get_array_of_samples()
audio = np.array(audio)
audio.shape

In [ ]:
import sksound
mySound = sksound.sounds.Sound(audio_file)
mySound.read_sound(audio_file)

In [ ]:
mySound.data

In [ ]:
display.Audio(data=mySound.data, rate=16000)

In [ ]:
count = 0
for audio_file in audio_files:
    audio_data, _ = librosa.load(sr=None, mono=True, path=audio_file)
    count += 1
    print(count)

In [ ]:
def vox2_val_to_wav():
    audio_files = glob("/data/techresearch/Murtaza/vox2/val/aac/*.m4a")
    export_dir = '/data/techresearch/Murtaza/vox2/val/wav/'
    count = 0
    for file in audio_files:
        file_name = os.path.split(file)[1]
        file_name = file_name.replace('.m4a','')
        audio = AudioSegment.from_file(file)
        audio.export(export_dir + file_name + '.wav', format='wav')
        count += 1
        if count%100 == 0:
            print(count)
    return

In [ ]:
def vox2_dev_to_wav():
    audio_files = glob("/data/techresearch/Murtaza/vox2/dev/aac/*/*.m4a")
    export_dir = '/data/techresearch/Murtaza/vox2/dev/wav/'
    count = 0
    for file in audio_files:
        dir_name = os.path.split(os.path.split(file)[0])[1]
        file_name = os.path.split(file)[1]
        file_name = file_name.replace('.m4a','')
        audio = AudioSegment.from_file(file)
        audio.export(export_dir + dir_name + '/' + file_name + '.wav', format='wav')
        count +=1
        if (count%100) == 0:
            print(count)
    return

In [ ]:
def name_correction():
    

In [ ]:
def shuffle_dataset(x,y):
    combined = list(zip(x,y))
    random.shuffle(combined)
    x, y = zip(*combined)
    x = list(x)
    y = list(y)
    return x, y

In [8]:
def create_model():
    '''
    Creates and returns a new model.
    '''
    
    #For Spectogram Input features (No convergence)
    '''
    model_input = Input(shape=(129,285,1))
    
    conv1 = Conv2D(filters=96, kernel_size=(7, 7), strides=(2, 2), activation='relu')(model_input)
    mpool1 = MaxPooling2D(pool_size=(3, 3), strides=(2,2))(conv1)
    
    conv2 = Conv2D(filters=256, kernel_size=(5, 5), strides=(2, 2), activation='relu')(mpool1)
    #mpool2 = MaxPooling2D(pool_size=(3, 3), strides=(2,2))(conv2)
    
    conv3 = Conv2D(filters=384, kernel_size=(3, 3), strides=(1, 1), activation='relu')(conv2)
    conv4 = Conv2D(filters=256, kernel_size=(3, 3), strides=(1, 1), activation='relu')(conv3)
    conv5 = Conv2D(filters=256, kernel_size=(3, 3), strides=(1, 1), activation='relu')(conv4)
    
    flatten = Flatten()(conv5)
    
    fc7 = Dense(1024)(flatten)
    fc8 = Dense(1200, activation='softmax')(fc7)
    
    model = Model(model_input, fc8)
    '''
    
    
    #For MFCC1 input features (Acc: 84, val_acc:70)
    '''
    model_input = Input(shape=(60,126,1))
    
    conv1 = Conv2D(filters=96, kernel_size=(7, 7), strides=(2, 2), activation='relu')(model_input)
    mpool1 = MaxPooling2D(pool_size=(3, 3), strides=(2,2))(conv1)
    
    conv2 = Conv2D(filters=256, kernel_size=(5, 5), strides=(2, 2), activation='relu')(mpool1)
    #mpool2 = MaxPooling2D(pool_size=(3, 3), strides=(2,2))(conv2)
    
    conv3 = Conv2D(filters=384, kernel_size=(3, 3), strides=(1, 1), activation='relu')(conv2)
    #conv4 = Conv2D(filters=256, kernel_size=(3, 3), strides=(1, 1), activation='relu')(conv3)
    #conv5 = Conv2D(filters=256, kernel_size=(3, 3), strides=(1, 1), activation='relu')(conv4)
    
    flatten = Flatten()(conv3)
    
    fc7 = Dense(1024)(flatten)
    fc8 = Dense(1200, activation='softmax')(fc7)
    
    model = Model(model_input, fc8)
    '''
    
    #For mfcc2 Input features (Good Convegence was witnessed)
    '''
    model_input = Input(shape=(332,300,1))
    
    conv1 = Conv2D(filters=96, kernel_size=(7, 7), strides=(2, 2), activation='relu')(model_input)
    mpool1 = MaxPooling2D(pool_size=(3, 3), strides=(2,2))(conv1)
    
    conv2 = Conv2D(filters=256, kernel_size=(5, 5), strides=(2, 2), activation='relu')(mpool1)
    mpool2 = MaxPooling2D(pool_size=(3, 3), strides=(2,2))(conv2)
    
    conv3 = Conv2D(filters=384, kernel_size=(3, 3), strides=(1, 1), activation='relu')(mpool2)
    conv4 = Conv2D(filters=256, kernel_size=(3, 3), strides=(1, 1), activation='relu')(conv3)
    conv5 = Conv2D(filters=256, kernel_size=(3, 3), strides=(1, 1), activation='relu')(conv4)
    
    flatten = Flatten()(conv5)
    
    fc7 = Dense(1024)(flatten)
    fc8 = Dense(1200, activation='softmax')(fc7)
    
    model = Model(model_input, fc8)
    '''
    
    
    #For mfcc3 Input features

    model_input = Input(shape=(512,299,1))
    #model_input_1 = BatchNormalization(scale=False, axis=3)(model_input)
    
    conv1 = Conv2D(filters=96, kernel_size=(7, 7), strides=(2, 2))(model_input)
    conv1 = BatchNormalization(scale=False, axis=3)(conv1)
    conv1 = Activation('relu')(conv1) 
    mpool1 = MaxPooling2D(pool_size=(3, 3), strides=(2,2))(conv1)
    
    conv2 = Conv2D(filters=256, kernel_size=(5, 5), strides=(2, 2))(mpool1)
    conv2 = BatchNormalization(scale=False, axis=3)(conv2)
    conv2 = Activation('relu')(conv2) 
    mpool2 = MaxPooling2D(pool_size=(3, 3), strides=(2,2))(conv2)
    
    conv3 = Conv2D(filters=384, kernel_size=(3, 3), strides=(1, 1))(mpool2)
    conv3 = BatchNormalization(scale=False, axis=3)(conv3)
    conv3 = Activation('relu')(conv3)
    
    conv4 = Conv2D(filters=256, kernel_size=(3, 3), strides=(1, 1))(conv3)
    conv4 = BatchNormalization(scale=False, axis=3)(conv4)
    conv4 = Activation('relu')(conv4) 
    
    conv5 = Conv2D(filters=256, kernel_size=(3, 3), strides=(1, 1))(conv4)
    conv5 = BatchNormalization(scale=False, axis=3)(conv5)
    conv5 = Activation('relu')(conv5)
    conv5 = Dropout(0.15)(conv5)
    mpool5 = MaxPooling2D(pool_size=(3, 3), strides=(2,2))(conv5)
    
    fc6 = Conv2D(filters=4096, kernel_size=(11, 1), strides=(1, 1))(mpool5)
    fc6 = BatchNormalization(scale=False, axis=3)(fc6)
    fc6 = Activation('relu')(fc6) 
    fc6 = Dropout(0.2)(fc6)
    apool6 = AveragePooling2D(pool_size=(1, 5), strides=(1,1))(fc6)
    
    flatten = Flatten()(apool6)
    
    fc7 = Dense(1024, activation='relu')(flatten)
    fc7 = Dropout(0.2)(fc7)
    fc8 = Dense(5994, activation='softmax')(fc7)
    
    model = Model(model_input, fc8)
    feature_model1 = Model(model_input, flatten)
    feature_model2 = Model(model_input, fc7)
    
    
    return model, feature_model1, feature_model2

### Feature Experiments:

In [ ]:
audio_data, _ = librosa.load(sr=16000, mono=True, path='/home/techresearch/rnn/hello.wav')
audio_data = audio_data[0:16000*3]
audio_mfcc1 = librosa.feature.mfcc(y=audio_data, sr=16000,  n_mfcc=300)

In [ ]:
audio_data, _ = librosa.load(sr=16000, mono=True, path='/home/techresearch/rnn/hello.wav')
audio_data = audio_data[0:16000*3]
audio_mfcc2 = mfcc(signal=audio_data, samplerate=16000, nfft=512, winlen=0.025, winstep=0.006, nfilt=400, numcep = 300, winfunc=np.hamming)

In [ ]:
audio_data, _ = librosa.load(sr=16000, mono=True, path='/home/techresearch/rnn/hello.wav')
audio_data = audio_data[0:16000*3]
audio_mfcc3 = get_fft_spectrum(audio_data)

In [ ]:
plt.imshow(audio_mfcc3)

In [ ]:
audio_mfcc4.shape

In [ ]:
display.Audio(data = audio_data, rate = 16000)

### Save Training Data:

In [ ]:
'''
#Load and shuffle dataset
x_train, y_train = load_training_dataset('/data/techresearch/Murtaza/vox1/dev/wav/', 600)
x_train, y_train = shuffle_dataset(x_train, y_train)
#Convert to numpy
x_train = np.asarray(x_train)
y_train = np.asarray(y_train)
#Save the numpy data
np.save('/data/techresearch/Murtaza/x_train.npy', x_train)
np.save('/data/techresearch/Murtaza/y_train.npy', y_train)
'''

### Train Model:

In [ ]:
def iterative_training(model, feature_model, no_epochs):
    for epoch in range(0,no_epochs):
        for batch in range(0,16):
            #Load the numpy Data from disk:
            x_train = np.load('/data/techresearch/Murtaza/mfcc3_gpu/x_train_' + str(batch) + '.npy')
            y_train = np.load('/data/techresearch/Murtaza/mfcc3_gpu/y_train_' + str(batch) + '.npy')
            #Reshape x_train to have 1 channel.
            x_train = np.reshape(x_train, [-1,512,299,1])
            #Onehot encoding for y_train.
            y_train = to_categorical(y_train)
            
            #Train the model.
            print('Epoch: ' + str(epoch + 1))
            print('Batch: ' + str(batch))
            model.fit(x=x_train, y=y_train, batch_size=64, epochs=1, validation_split=0.03)
            
        model.save('/data/techresearch/Murtaza/mfcc3_gpu/weights/model_' + str(epoch+1) + '.h5')
        feature_model.save('/data/techresearch/Murtaza_gpu/mfcc3/weights/feature_' + str(epoch+1) + '.h5')
        model.save_weights('/data/techresearch/Murtaza_gpu/mfcc3/weights/weights_' + str(epoch+1) + '.h5')

In [ ]:
#Prepare the model.
model, feature_model1, feature_model2 = create_model()
adam = keras.optimizers.Adam(lr=0.0001)
model.compile(optimizer=adam, loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

In [ ]:
#Change Learning Rate
#K.set_value(model.optimizer.lr, 0.0001)

In [ ]:
#iterative_training(model, feature_model, 3)
vox2_iterative_generative_training(model)

### Test Model:

In [ ]:
model = load_model('/data/techresearch/Murtaza/mfcc3/weights/model_7.h5')

In [ ]:
x_train = np.load('/data/techresearch/Murtaza/vox2/val/x_val.npy')
y_train = np.load('/data/techresearch/Murtaza/vox2/val/y_val.npy')
#Reshape x_train to have 1 channel.
x_train = np.reshape(x_train, [-1,512,299,1])
#Onehot encoding for y_train.
y_train = to_categorical(y_train)

In [ ]:
a = model.evaluate(x=x_train, y=y_train) 

### Enroll:

In [24]:
feature_model = load_model('/data/techresearch/Murtaza/vox2/dev/weights4/model_4_35.h5')
feature_model.outputs = [feature_model.layers[-2].output]
feature_model.layers[-2].outbound_nodes = []

In [10]:
x_enroll, y_enroll = create_enrollment_dataset()

No. Speakers loaded: 1
No. Speakers loaded: 2
No. Speakers loaded: 3
No. Speakers loaded: 4
No. Speakers loaded: 5
No. Speakers loaded: 6
No. Speakers loaded: 7
No. Speakers loaded: 8
No. Speakers loaded: 9
No. Speakers loaded: 10
No. Speakers loaded: 11
No. Speakers loaded: 12
No. Speakers loaded: 13
No. Speakers loaded: 14
No. Speakers loaded: 15
No. Speakers loaded: 16
No. Speakers loaded: 17
No. Speakers loaded: 18
No. Speakers loaded: 19
No. Speakers loaded: 20
No. Speakers loaded: 21
No. Speakers loaded: 22
No. Speakers loaded: 23
No. Speakers loaded: 24
No. Speakers loaded: 25
No. Speakers loaded: 26
No. Speakers loaded: 27
No. Speakers loaded: 28
No. Speakers loaded: 29
No. Speakers loaded: 30
No. Speakers loaded: 31
No. Speakers loaded: 32
No. Speakers loaded: 33
No. Speakers loaded: 34
No. Speakers loaded: 35
No. Speakers loaded: 36
No. Speakers loaded: 37
No. Speakers loaded: 38
No. Speakers loaded: 39
No. Speakers loaded: 40
No. Speakers loaded: 41
No. Speakers loaded: 42
N

In [25]:
x_enroll = np.reshape(x_enroll, [-1,512,299,1])
y_enroll = np.reshape(y_enroll, [-1,1])
features_enroll = feature_model.predict(x_enroll)

In [26]:
svc = SVC(probability=True, kernel='rbf', gamma='scale')
svc.fit(features_enroll, y_enroll)

/home/techresearch/.local/lib/python3.6/site-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
  max_iter=-1, probability=True, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

### Eval

In [27]:
x_eval, y_eval = create_evaluation_dataset()
x_eval = np.reshape(x_eval, [-1,512,299,1])
y_eval = np.reshape(y_eval, [-1,1])

In [28]:
y_eval.shape

(31, 1)

In [29]:
features_eval = feature_model.predict(x_eval)
predictions_eval = svc.predict(features_eval)
predictions_eval = np.reshape(predictions_eval, (-1,1))
unique, counts = np.unique((predictions_eval == y_eval), return_counts=True)
results_eval = dict(zip(unique, counts))

In [30]:
thresh = 0.05
result_with_thresh = []

probabilities = svc.predict_proba(features_eval)

for i, pred in enumerate(predictions_eval):
    index = np.where(svc.classes_==pred)[0][0]
    if probabilities[i][index] > thresh:
        if pred == y_eval[i]:
            result_with_thresh.append(True)
        else:
            result_with_thresh.append(False)
    else:
        result_with_thresh.append('Rejected')
        
f = result_with_thresh.count(False)
t = result_with_thresh.count(True)
r = result_with_thresh.count('Rejected')

print(results_eval)
accuracy_eval = float(results_eval[True])/len(predictions_eval)
print('Accuracy: ' + str(accuracy_eval))

acc = t/(t+f)
print('Rejected: ' + str(r))
print('Thresh: ' + str(acc))

{False: 13, True: 18}
Accuracy: 0.5806451612903226
Rejected: 11
Thresh: 0.75
